In [1]:
!pip install transformers trl peft sentence-transformers accelerate opendatasets wandb -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 154.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 2

In [2]:
!pip install pytorch-lightning faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 73.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 106.9 MB/s eta 0:00:00


In [24]:
import torch
import torch.nn.functional as F
from sentence_transformers.util import cos_sim

/usr/local/lib/python3.11/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-24 22:25:24.893875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 22:25:24.893948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 22:25:24.895014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 22:25:24.901365: I tensorflow/core/platform/cpu_feature_guard.cc:18

In [25]:
t1 = torch.rand(32,1024)
t2 = torch.rand(128,1024)

In [47]:
torch.mm(t1,t2.T).shape

torch.Size([32, 128])

In [1]:
!nvidia-smi

Wed Sep 25 11:22:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 58%   82C    P2   297W / 300W |  29023MiB / 49140MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
def check_idxs(idxs,df,negs,poss,batch_size=8):
        for i in range(0,len(df),batch_size):
            idx_batch = idxs[i*batch_size:(i+1)*batch_size]
            batch_pos = [poss[j] for j in idx_batch]
            batch_neg = []
            for j in idx_batch:
                batch_neg.extend(negs[j])
            all_states = batch_pos + batch_neg
            for ps in batch_pos:
                if all_states.count(ps) != 1:
                    return False
        return True

In [3]:
import pandas as pd
df = pd.read_parquet('candidates_bge_large.parquet')

In [7]:
def parse_negs(x):
    return [i for i in x['top25_candidates'] if i != x['misconcpts_id']][:5]

In [16]:
poss = df['misconcpts_id'].tolist()

In [8]:
negs = df.apply(parse_negs,axis=1).tolist()

In [28]:
negs[0]

[2398, 979, 1593, 1540, 363]

In [58]:
import random
from tqdm.auto import trange

In [55]:
idxs = list(range(len(df)))
random.shuffle(idxs)

In [64]:
for ii in trange(10_000_000):
    random.shuffle(idxs)
    if check_idxs(idxs,df,negs,poss,batch_size=8):
        print('donde')
        break

  0%|          | 0/10000000 [00:00<?, ?it/s]

donde


In [65]:
idxs

[2027,
 1284,
 1747,
 3826,
 454,
 3017,
 642,
 2090,
 2757,
 5,
 304,
 2220,
 3170,
 1946,
 2544,
 3253,
 1290,
 4088,
 2510,
 906,
 2953,
 1100,
 2141,
 2271,
 1871,
 475,
 3587,
 1381,
 680,
 776,
 1134,
 4301,
 2219,
 2490,
 3111,
 523,
 284,
 2444,
 2379,
 3623,
 1050,
 4107,
 163,
 2912,
 3146,
 2565,
 1202,
 2338,
 3040,
 3759,
 3901,
 1103,
 1914,
 153,
 1530,
 2739,
 4056,
 4315,
 3590,
 3119,
 3554,
 4191,
 1014,
 3701,
 3096,
 4355,
 984,
 1075,
 2043,
 2429,
 219,
 3004,
 505,
 2475,
 4106,
 4287,
 2321,
 1761,
 2961,
 3550,
 884,
 1886,
 2181,
 666,
 647,
 3102,
 1024,
 2759,
 812,
 4002,
 1521,
 1074,
 3094,
 1358,
 3936,
 2850,
 1380,
 1219,
 3478,
 1398,
 568,
 549,
 4017,
 2607,
 374,
 2356,
 1789,
 4281,
 616,
 3995,
 2588,
 1083,
 42,
 3707,
 3880,
 3791,
 1135,
 2355,
 4283,
 4181,
 3305,
 1507,
 58,
 2088,
 1141,
 656,
 1936,
 3511,
 120,
 469,
 1587,
 3578,
 2439,
 3985,
 1616,
 2728,
 3034,
 1184,
 599,
 449,
 3551,
 3236,
 422,
 2489,
 2903,
 4227,
 3716,
 60,
 

In [9]:
import os

In [12]:
os.listdir('.')

['wandb',
 'candidates_bge_large.csv',
 'src',
 'candidates_bge_large.parquet',
 'bge_val.ipynb',
 'fold_0.npy',
 'Gemma.ipynb',
 '.git',
 'bge_candidate_gen.ipynb',
 'eedi-mining-misconceptions-in-mathematics',
 'simple_trained_encoder.zip',
 'load_data.ipynb',
 'quick_start_pytorch.ipynb',
 '.ipynb_checkpoints',
 'train_bge_ce.ipynb']